<a href="https://colab.research.google.com/github/CarlosCordova2023/ML_M01_1/blob/main/ML_M01_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Grafo bipartito simple: usuarios 'u:*' e ítems 'i:*'
from collections import deque  # Cola FIFO eficiente

# Adyacencias: para cada nodo, a qué nodos se conecta
G = {
    "u:ana": {"i:book1", "i:book2"},
    "u:luis": {"i:book2", "i:book3"},
    "u:sol": {"i:book3"},
    "i:book1": {"u:ana"},
    "i:book2": {"u:ana", "u:luis"},
    "i:book3": {"u:luis", "u:sol"},
}

def bfs_levels(graph, source, max_depth=2):
    # Nivel (distancia en saltos) de cada nodo visitado
    level = {source: 0}         # El origen está a distancia 0 de sí mismo
    # Cola para explorar vecinos en orden creciente de nivel (BFS)
    q = deque([source])         # Comenzamos en 'source'
    while q:                    # Mientras haya nodos por procesar
        v = q.popleft()         # Saco el más antiguo (FIFO)
        if level[v] == max_depth:  # Si ya alcancé la profundidad deseada
            continue            # No expandir más desde aquí
        for w in graph.get(v, ()):  # Recorro vecinos de v
            if w not in level:      # Si aún no fue visitado
                level[w] = level[v] + 1  # Su nivel es el de v + 1
                q.append(w)              # Encolarlo para explorar sus vecinos
    return level                 # Devuelvo niveles por nodo

def recommend_2_hops(graph, user):
    # Ejecutamos BFS hasta 2 saltos desde 'user'
    levels = bfs_levels(graph, user, max_depth=2)  # Distancias por nodo
    # Candidatos: ítems a distancia 2 (u -> i -> u -> i) o 1 (u -> i) si quisieras
    recs = [n for n, d in levels.items() if n.startswith("i:") and d == 2]
    # Filtramos ítems que ya tiene el usuario (nivel 1)
    owned = {n for n, d in levels.items() if n.startswith("i:") and d == 1}
    return [r for r in recs if r not in owned]     # Sugerir lo nuevo

# Ejemplo: recomendaciones para Ana
#print(recommend_2_hops(G, "u:ana"))  # Esperable: ['i:book3']
print(recommend_2_hops(G, "u:luis"))  # Esperable: ['i:book3']


[]


In [6]:
import heapq  # Cola de prioridad (mínimo primero)

# Grafo ponderado: dict[nodo] -> lista de (vecino, costo)
W = {
    "A": [("B", 2), ("C", 5)],
    "B": [("A", 2), ("C", 1), ("D", 3)],
    "C": [("A", 5), ("B", 1), ("D", 1)],
    "D": [("B", 3), ("C", 1)],
}

def dijkstra(graph, source):
    # dist almacena la mejor distancia conocida a cada nodo
    dist = {source: 0.0}                   # Al origen se llega con costo 0
    # heap almacena pares (distancia acumulada, nodo)
    heap = [(0.0, source)]                 # Inicializo con el origen
    visited = set()                        # Para no re-procesar nodos fijos
    while heap:                            # Mientras haya candidatos
        d, v = heapq.heappop(heap)         # Tomo el nodo con menor distancia
        if v in visited:                   # Si ya lo fijé, salto
            continue
        visited.add(v)                     # Lo marco como procesado
        for w, cost in graph.get(v, []):   # Exploro vecinos
            nd = d + cost                  # Costo alternativo por v -> w
            if nd < dist.get(w, float("inf")):  # Si mejora lo conocido
                dist[w] = nd               # Actualizo la mejor distancia
                heapq.heappush(heap, (nd, w))  # Reencolo w con su nuevo costo
    return dist                             # Distancias finales desde source

# Feature: distancia mínima al hub 'A'
dist_to_hub = dijkstra(W, "A")             # Ejecuta Dijkstra desde 'A'
print(dist_to_hub)                         # {'A': 0.0, 'B': 2.0, 'C': 3.0, 'D': 4.0}
# Estas distancias pueden añadirse como columnas de features en tu dataset tabular.


{'A': 0.0, 'B': 2.0, 'C': 3.0, 'D': 4.0}


In [7]:
import numpy as np  # Cálculo vectorizado

def gini_impurity(y):
    # Calcula 1 - sum(p_k^2) para clases en y
    m = len(y)                                    # N° de muestras
    if m == 0: return 0.0                         # Caso vacío
    _, counts = np.unique(y, return_counts=True)  # Frecuencias por clase
    p = counts / m                                # Probabilidades
    return 1.0 - np.sum(p * p)                    # Fórmula de Gini

def best_threshold_stump(x, y):
    # Devuelve (umbral, pred_left, pred_right, score) que minimiza Gini ponderado
    order = np.argsort(x)                         # Índices que ordenan x
    x_sorted, y_sorted = x[order], y[order]       # Reordenamos por x
    # Candidatos: promedios entre valores consecutivos
    thr_candidates = (x_sorted[:-1] + x_sorted[1:]) / 2.0
    best = (None, None, None, float("inf"))       # Mejor (thr, yL, yR, score)
    for thr in thr_candidates:                    # Probar cada umbral
        left = y_sorted[x_sorted <= thr]          # Etiquetas a la izquierda
        right = y_sorted[x_sorted > thr]          # Etiquetas a la derecha
        # Gini ponderado por tamaño de cada lado
        m = len(y_sorted)
        score = (len(left)/m)*gini_impurity(left) + (len(right)/m)*gini_impurity(right)
        if score < best[3]:                       # Si mejora el mejor actual
            # La predicción en cada lado será la clase mayoritaria
            pred_left = np.bincount(left).argmax()
            pred_right = np.bincount(right).argmax()
            best = (thr, pred_left, pred_right, score)
    return best                                    # Devuelvo el mejor stump

def predict_stump(x, stump):
    # Aplica el stump: si x <= thr -> pred_left, si no -> pred_right
    thr, pred_left, pred_right, _ = stump         # Desempaquetar parámetros
    return np.where(x <= thr, pred_left, pred_right)  # Vector de predicciones

# Datos de ejemplo (binarios): una sola feature 'x' y etiquetas {0,1}
x = np.array([0.2, 0.4, 0.5, 0.55, 0.8, 0.85])     # Valores escalares
y = np.array([0,   0,   0,   1,    1,    1])       # Clases binarias

stump = best_threshold_stump(x, y)                  # Entreno el “árbol” de 1 split
print("Mejor stump:", stump)                        # (umbral, yL, yR, gini)
print("Predicciones:", predict_stump(x, stump))     # Aplico el stump al conjunto


Mejor stump: (np.float64(0.525), np.int64(0), np.int64(1), np.float64(0.0))
Predicciones: [0 0 0 1 1 1]
